# Processing data for modelling

In [1]:
import pandas as pd
# test with feature_engineering.py
prices = pd.read_csv('data/train_files/stock_prices.csv', parse_dates=[1])
print(prices.shape)
prices.head(2)


(2332531, 12)


,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324


In [33]:

from feature_engineering import fill_and_drop_na_values

fill_prices = fill_and_drop_na_values(prices)
print(fill_prices.shape)
fill_prices.head(2)


100%|██████████| 2000/2000 [11:06<00:00,  3.00it/s]


(2332531, 12)


,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20170104_1301,2017-01-04 00:00:00,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.00073
1865,20170105_1301,2017-01-05 00:00:00,1301,2743.0,2747.0,2735.0,2738.0,17900,1.0,NaN,False,0.00292


In [37]:
fill_prices.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2332531 entries, 0 to 2331102
Data columns (total 12 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   RowId             object
 1   Date              object
 2   SecuritiesCode    object
 3   Open              object
 4   High              object
 5   Low               object
 6   Close             object
 7   Volume            object
 8   AdjustmentFactor  object
 9   ExpectedDividend  object
 10  SupervisionFlag   object
 11  Target            object
dtypes: object(12)
memory usage: 231.3+ MB


In [39]:
from datetime import datetime
#fill_prices['Date'] = fill_prices['Date'].apply(lambda x: datetime.strptime(str(x),'%Y-%m-%d'))
fill_prices['Date'] = pd.to_datetime(fill_prices['Date'])

In [40]:
fill_prices.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2332531 entries, 0 to 2331102
Data columns (total 12 columns):
 #   Column            Dtype         
---  ------            -----         
 0   RowId             object        
 1   Date              datetime64[ns]
 2   SecuritiesCode    object        
 3   Open              object        
 4   High              object        
 5   Low               object        
 6   Close             object        
 7   Volume            object        
 8   AdjustmentFactor  object        
 9   ExpectedDividend  object        
 10  SupervisionFlag   object        
 11  Target            object        
dtypes: datetime64[ns](1), object(11)
memory usage: 231.3+ MB


In [3]:
from feature_engineering import adjust_price

adjusted_price = adjust_price(fill_prices)
print(adjusted_price.shape)
adjusted_price.head(2)

100%|██████████| 2000/2000 [17:59<00:00,  1.85it/s]

(2332531, 18)


,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,ad_Open,ad_High,ad_Low,ad_Close,ad_Volume,ad_Target
1483,20170104_8194,2017-01-04 00:00:00,8194,3325.0,3390.0,3300.0,3380.0,36100,1.0,NaN,False,-0.002963,3325.0,3390.0,3300.0,3380.0,36100.0,-0.002963
3348,20170105_8194,2017-01-05 00:00:00,8194,3395.0,3420.0,3360.0,3375.0,19400,1.0,NaN,False,-0.020802,3395.0,3420.0,3360.0,3375.0,19400.0,-0.020802


In [41]:
adjusted_price['Date'] = pd.to_datetime(adjusted_price['Date'])

In [4]:
#from feature_engineering import fill_finances

financial = financial = pd.read_csv('data/train_files/financials.csv',parse_dates=[2])
financial.head(2)

,DisclosureNumber,DateCode,Date,SecuritiesCode,DisclosedDate,DisclosedTime,DisclosedUnixTime,TypeOfDocument,CurrentPeriodEndDate,TypeOfCurrentPeriod,...,ForecastEarningsPerShare,ApplyingOfSpecificAccountingOfTheQuarterlyFinancialStatements,MaterialChangesInSubsidiaries,ChangesBasedOnRevisionsOfAccountingStandard,ChangesOtherThanOnesBasedOnRevisionsOfAccountingStandard,ChangesInAccountingEstimates,RetrospectiveRestatement,NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock,NumberOfTreasuryStockAtTheEndOfFiscalYear,AverageNumberOfShares
0,2.016121e+13,20170104_2753,2017-01-04,2753.0,2017-01-04,07:30:00,1.483483e+09,3QFinancialStatements_Consolidated_JP,2016-12-31,3Q,...,319.76,NaN,False,True,False,False,False,6848800.0,－,6848800.0
1,2.017010e+13,20170104_3353,2017-01-04,3353.0,2017-01-04,15:00:00,1.483510e+09,3QFinancialStatements_Consolidated_JP,2016-11-30,3Q,...,485.36,NaN,False,True,False,False,False,2035000.0,118917,1916083.0


In [22]:
from feature_engineering import fill_finances_knn



filled_finances = fill_finances_knn(financial, prices)
filled_finances.head()
filled_finances.shape

100%|██████████| 2000/2000 [00:06<00:00, 302.48it/s]


(48142, 7)

In [43]:
filled_finances.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48142 entries, 0 to 6
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Date            48142 non-null  datetime64[ns]
 1   Day             48142 non-null  float64       
 2   Month           48142 non-null  float64       
 3   Year            48142 non-null  float64       
 4   SecuritiesCode  48142 non-null  float64       
 5   Profit          48142 non-null  float64       
 6   NetSales        48142 non-null  float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 2.9 MB


In [44]:
# concat financials and price
price_financial = pd.merge(adjusted_price, filled_finances, how='left', on=['SecuritiesCode', 'Date'])

In [45]:
price_financial.shape

(2335601, 23)

In [10]:
from tqdm import tqdm

In [49]:
# ffill and drop 
# TODO Drop expectedDividend?
#sec_codes = price_financial.SecuritiesCode.unique()
sec_codes = price_financial.SecuritiesCode.unique()
#a = pd.merge(prices, financial, how='left', on='SecuritiesCode')

price_financial_fill = pd.DataFrame(columns=price_financial.columns)
for i in tqdm(sec_codes):

    current = price_financial.query('SecuritiesCode == @i')
    current = current.ffill()
    current = current.dropna(axis=0)

    price_financial_fill  = pd.concat([price_financial_fill , current])

100%|██████████| 2000/2000 [26:27<00:00,  1.26it/s]


In [50]:
# new features
#price_financial_fill.drop_duplicates(inplace=True)
price_financial_fill.shape

(2205125, 23)

In [ ]:
# save dataframe to csv